In [1]:
!gdown --id 15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
#gets labeled data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
To: /content/earning_annotation.csv
100% 98.8k/98.8k [00:00<00:00, 98.7MB/s]


In [2]:
import pandas as pd, csv, numpy as np
df=pd.read_csv('earning_annotation.csv')

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!nvidia-smi

Wed Jul 20 13:58:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import torch
from tqdm.notebook import tqdm
from transformers import  AlbertTokenizer, AlbertModel
from torch.utils.data import TensorDataset

from transformers import AlbertTokenizer, AlbertForSequenceClassification
import pandas as pd

In [6]:
N = 7
# Select first N columns
df = df.iloc[: , :N]

In [7]:
df.head(20)

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
5,MSFT_q4_2018.txt,Microsoft 365 is now a multi-billion-dollar bu...,"['cloud', 'the cloud']",['Cloud computing'],Pres,Pres,1
6,MSFT_q4_2018.txt,"Across Microsoft 365, we are helping people be...",['AI'],['AI'],Pres,Pres,1
7,MSFT_q4_2018.txt,Outlook mobile is being used on more than 100 ...,['mobile'],['Mobile'],Pres,Pres,1
8,MSFT_q4_2018.txt,This strong engagement is driven by quality of...,['mobile'],['Mobile'],Pres,Pres,1
9,MSFT_q4_2018.txt,We will continue to invest to make LinkedIn th...,['platform'],['Digital Product'],Fut,Pres,1


In [8]:
df=df.head(401)

In [9]:
df

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
...,...,...,...,...,...,...,...
396,IBM_q1_2018.txt,We’ve grown to over 50 active blockchain netwo...,"['platform', 'blockchain']","['Blockchain', 'Digital Product']",Pres,Pres,1
397,IBM_q1_2018.txt,"And last month, we announced a beta version of...","['platform', 'blockchain']","['Blockchain', 'Digital Product']",Past,Past,0
398,IBM_q1_2018.txt,Our strategic imperatives revenue reflects the...,['cloud'],['Cloud computing'],Pres,Pres,1
399,IBM_q1_2018.txt,Revenue growth was led by strong double-digit ...,"['transformation', 'digital transformation', '...","['Practices', 'Enablers']",Past,Past,0


In [10]:
from sklearn.model_selection import train_test_split, KFold
def split_for_kfold(x, y, kd):
    kfold = KFold(n_splits=kd, random_state=0, shuffle=True)
    for train_val_index, test_index in kfold.split(x):
        # Get train_val (K-1 folds) and test (1 fold)
        x_train_val, x_test = x[train_val_index], x[test_index]
        y_train_val, y_test = y[train_val_index], y[test_index] 
        
        # Partition the train_val set
        x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=42)
    return x_train, x_val, x_test, y_train, y_val, y_test

In [11]:
# X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
X_train, X_val, X_test, y_train, y_val, y_test = split_for_kfold(df.index.values,
                                                                 df.label.values,
                                                                 5)

In [12]:
# from sklearn.model_selection import train_test_split

# X_train, X_rem, y_train, y_rem = train_test_split(df.index.values, 
#                                                   df.label.values, 
#                                                   train_size=0.75, 
#                                                   random_state=42, 
#                                                   stratify=df.label.values)


In [13]:
X_test

array([  9,  25,  28,  31,  32,  38,  39,  42,  47,  53,  57,  70,  72,
        82,  84,  87,  88,  91,  99, 105, 115, 117, 119, 121, 127, 128,
       131, 147, 151, 163, 165, 172, 174, 177, 183, 185, 192, 193, 195,
       197, 202, 203, 211, 242, 243, 244, 251, 257, 262, 265, 267, 273,
       277, 285, 288, 290, 291, 292, 314, 321, 323, 324, 333, 335, 337,
       338, 346, 348, 354, 356, 358, 359, 363, 367, 369, 371, 373, 385,
       387, 399])

In [14]:
y_train

array([0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 2, 1, 2, 2, 2, 1, 1, 1, 2, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1,
       0, 0, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 2, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 2, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 2, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 1,
       2, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 2, 2, 1, 1, 0, 1, 1, 1, 0, 2, 1,
       1, 2, 1, 0, 0, 2, 0, 1, 0, 2, 2, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 2,
       1, 1, 2, 0, 2, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 2, 2])

In [15]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.loc[X_test, 'data_type']= 'test'

In [16]:
df.groupby(['label', 'data_type']).count()

id  text  gone  gtwo  machine  human
label data_type                                       
0     test        35    35    35    35       35     35
      train       83    83    83    83       83     83
      val         24    24    24    24       24     24
1     test        30    30    30    30       30     30
      train      121   121   121   121      118    121
      val         45    45    45    45       45     45
2     test        15    15    15    15       15     15
      train       36    36    36    36       35     36
      val         12    12    12    12       12     12

In [17]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')


Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [19]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test= TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [21]:
len(dataset_train), len(dataset_val), len(dataset_test)

(240, 81, 80)

In [22]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2',
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [23]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [24]:
from transformers import AdamW, get_linear_schedule_with_warmup

# optimizer = AdamW(model.parameters(),
#                   lr=1e-5, 
#                   eps=1e-8)
optimizer = torch.optim.AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [25]:
epochs = 6

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [26]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        # print(f'Class: {label_dict_inverse[label]}')
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [27]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [29]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def predict(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # loss = outputs[0]
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        
    predictions = np.concatenate(predictions, axis=0)            
    return predictions

In [30]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/80 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0336847309023143
Validation loss: 0.8645330810988391
F1 Score (Weighted): 0.6099198613818496


Epoch 2:   0%|          | 0/80 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.7553616302087903
Validation loss: 0.6479543756555628
F1 Score (Weighted): 0.658480227107678


...


Epoch 4
Training loss: 0.30757249501766637
Validation loss: 0.4570688088596971
F1 Score (Weighted): 0.8768322391510798


Epoch 5:   0%|          | 0/80 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.22477106733713298
Validation loss: 0.5154142924239514
F1 Score (Weighted): 0.8644145718016146


Epoch 6:   0%|          | 0/80 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.1958250959200086
Validation loss: 0.562117412496634
F1 Score (Weighted): 0.8644145718016146


In [31]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [32]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 20/24

Class: 1
Accuracy: 39/45

Class: 2
Accuracy: 11/12

